In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('../input/hivprogression/training_data.csv')
test_data = pd.read_csv('../input/hivprogression/test_data.csv')

In [ ]:
le = preprocessing.LabelEncoder()
df['PR Seq'] = le.fit_transform(df['PR Seq'].astype(str))
df['RT Seq'] = le.fit_transform(df['RT Seq'].astype(str))
test_data['PR Seq'] = le.fit_transform(test_data['PR Seq'].astype(str))
test_data['RT Seq'] = le.fit_transform(test_data['RT Seq'].astype(str))

In [ ]:
y=df['Resp'].values
x=df.drop(['Resp', 'PatientID'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3,random_state=3, stratify=y)

In [ ]:
multinb_model = MultinomialNB()
multinb_model.fit(X_train,Y_train)
Y_pred_multinb = multinb_model.predict(X_test)


In [ ]:
precision = precision_score(Y_test, Y_pred_multinb, average='binary')
recall = recall_score(Y_test, Y_pred_multinb, average='binary')
f1score = f1_score(Y_test, Y_pred_multinb, average='binary')
print(precision)
print(recall)
print(f1score)


In [ ]:
print(classification_report(Y_test,Y_pred_multinb))

In [ ]:
#plotting the roc curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(Y_test,Y_pred_multinb)
fpr, tpr, _ = roc_curve(Y_test,Y_pred_multinb)
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, marker='.',color='orange',label="for Multinomial Naive Bayes, auc = %.2f"% roc_auc)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, Y_train)
predicted = rfc_model.predict(X_test)
print('Random Forest:', accuracy_score(Y_test, predicted))

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
test=test_data[['PR Seq', 'RT Seq', 'VL-t0', 'CD4-t0']]
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print('Random Forest Classifier:', accuracy_score(Y_test, predicted))
predict = random_forest.predict(test)

In [ ]:
test=test_data[['PR Seq', 'RT Seq', 'VL-t0', 'CD4-t0']]
predict = random_forest.predict(test)

In [ ]:
submissions = pd.DataFrame({
    'PatientID': np.arange(1,predict.shape[0]+1),
    'ResponderStatus': predict
})
submissions.to_csv('submission.csv', index=False)